# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
from IPython.display import Image

# Import API key
from api_keys import geoapify_key

base_url = "https://maps.geoapify.com/v1/staticmap"

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,23.08,100,100,2.98,EC,2024-05-12 10:32:55
1,1,uturoa,-16.7333,-151.4333,27.37,74,5,6.26,PF,2024-05-12 10:35:14
2,2,martinsville,36.6915,-79.8725,10.25,91,2,2.15,US,2024-05-12 10:38:46
3,3,port alfred,-33.5906,26.8910,19.54,64,2,3.13,ZA,2024-05-12 10:33:58
4,4,xinyuan,37.3000,99.0000,12.59,19,7,6.08,CN,2024-05-12 10:38:46


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit the given criteria and drop any results with null values. Criteria: max temperature lower than 27 degrees but higher than 21, wind speed 
#less than 4.5 m/s, and cloudiness equals 0
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df_nonull = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df_nonull

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,atasu,48.6814,71.6433,21.68,22,0,1.98,KZ,2024-05-12 10:39:02
84,84,haiku-pauwela,20.9219,-156.3051,21.13,85,0,2.68,US,2024-05-12 10:33:27
131,131,vila velha,-20.3297,-40.2925,23.97,94,0,1.03,BR,2024-05-12 10:33:16
154,154,mahina,-17.5065,-149.4890,25.87,73,0,2.57,PF,2024-05-12 10:39:21
221,221,sirte,31.2089,16.5887,22.08,57,0,4.24,LY,2024-05-12 10:39:38
241,241,masallatah,32.6167,14.0000,23.72,39,0,2.36,LY,2024-05-12 10:39:43
277,277,hebi,35.8992,114.1925,24.84,15,0,2.03,CN,2024-05-12 10:39:52
288,288,luz,-19.8014,-45.6856,21.50,62,0,1.60,BR,2024-05-12 10:39:54
305,305,saint-francois,46.4154,3.9054,23.01,54,0,2.57,FR,2024-05-12 10:34:29
334,334,linda,39.1277,-121.5508,21.82,69,0,2.57,US,2024-05-12 10:40:08


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df_nonull[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
73,atasu,KZ,48.6814,71.6433,22,
84,haiku-pauwela,US,20.9219,-156.3051,85,
131,vila velha,BR,-20.3297,-40.2925,94,
154,mahina,PF,-17.5065,-149.4890,73,
221,sirte,LY,31.2089,16.5887,57,
241,masallatah,LY,32.6167,14.0000,39,
277,hebi,CN,35.8992,114.1925,15,
288,luz,BR,-19.8014,-45.6856,62,
305,saint-francois,FR,46.4154,3.9054,54,
334,linda,US,39.1277,-121.5508,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
        'categories': 'accommodation.hotel',
        'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atasu - nearest hotel: Индира
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
vila velha - nearest hotel: Hostess Hotel
mahina - nearest hotel: Hôtel Sarah Nui
sirte - nearest hotel: فندق أفريقيا سرت
masallatah - nearest hotel: No hotel found
hebi - nearest hotel: No hotel found
luz - nearest hotel: Hotel Excalibur
saint-francois - nearest hotel: Chez Lily
linda - nearest hotel: Hard Rock Hotel & Casino Sacramento at Fire Mountain
shizuishan - nearest hotel: No hotel found
rumilly - nearest hotel: Auberge de Motz
arraial do cabo - nearest hotel: Pousada Vida Boa Praia
cockburn town - nearest hotel: The Salt Raker Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
73,atasu,KZ,48.6814,71.6433,22,Индира
84,haiku-pauwela,US,20.9219,-156.3051,85,Inn At Mama's Fish House
131,vila velha,BR,-20.3297,-40.2925,94,Hostess Hotel
154,mahina,PF,-17.5065,-149.4890,73,Hôtel Sarah Nui
221,sirte,LY,31.2089,16.5887,57,فندق أفريقيا سرت
241,masallatah,LY,32.6167,14.0000,39,No hotel found
277,hebi,CN,35.8992,114.1925,15,No hotel found
288,luz,BR,-19.8014,-45.6856,62,Hotel Excalibur
305,saint-francois,FR,46.4154,3.9054,54,Chez Lily
334,linda,US,39.1277,-121.5508,69,Hard Rock Hotel & Casino Sacramento at Fire Mo...


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)